In [6]:
"""
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
tokenizer = AutoTokenizer.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")
ner("Metin burada girilir")



dbmdz/distilbert-base-turkish-cased
"""
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                          DataCollatorForTokenClassification, TrainingArguments, Trainer)
from datasets import load_dataset

In [7]:
label_list = ["O","B-IL","I-IL","B-ILCE","I-ILCE","B-MAH","I-MAH","B-SOK","I-SOK","B-NO","I-NO","B-PK","I-PK"]
id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in enumerate(label_list)}

In [8]:
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id
)

# Örnek: Hugging Face Datasets formatında token/labels kolonları olduğunu varsayalım
dataset = load_dataset("json", data_files={"train":"train.json","validation":"valid.json"})

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: Unable to find '/Users/bariscakmak/Desktop/localhost/playground/train.json'

In [9]:
def tokenize_align(example):
    toks = tokenizer(example["tokens"], is_split_into_words=True,
                     truncation=True, padding=False, max_length=128)
    word_ids = toks.word_ids()
    labels = []
    i = 0
    for wid in word_ids:
        if wid is None:
            labels.append(-100)                 # loss'a katılmasın
        else:
            labels.append(example["labels"][wid])
    toks["labels"] = labels
    return toks

In [10]:
dataset = dataset.map(tokenize_align, batched=False)
data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(
    output_dir="ner_tr_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True
)

trainer = Trainer(model=model, args=args,
                  train_dataset=dataset["train"],
                  eval_dataset=dataset["validation"],
                  tokenizer=tokenizer,
                  data_collator=data_collator)

trainer.train()

NameError: name 'dataset' is not defined